In [1]:
from kafka.admin import KafkaAdminClient, NewTopic


admin_client = KafkaAdminClient(
    bootstrap_servers="localhost:9092", 
    client_id='tracking'
)

In [1]:
import json
import folium
import time
import re
import requests
from google.transit import gtfs_realtime_pb2
from json import dumps
from kafka import KafkaProducer
from kafka import KafkaConsumer

api_url = 'https://realtime.hsl.fi/realtime/vehicle-positions/v2/hsl'  # Replace with your API endpoint
KAFKA_SERVER = 'localhost:9092'
TOPIC = 'coordinates'


def fetch_coordinates():
    response = requests.get(api_url)
    if response.status_code == 200:
        # Fetch the GTFS-RT payload
        response = requests.get(api_url)
        payload = response.content

        # Create a GTFS-RT feed message
        feed = gtfs_realtime_pb2.FeedMessage()

        # Parse and decode the payload
        feed.ParseFromString(payload)

        # Create a list to hold the decoded entities
        decoded_entities = []

        # Iterate over the feed entities
        for entity in feed.entity:
            # Convert the entity to a dictionary
            # 18/165 47/668
            if re.search(r'18/165', entity.id):
                entity_dict = {
                    'id': entity.id,
                    'vehicle': {
                        'trip': {
                            'route_id': entity.vehicle.trip.route_id,
                            'start_time': entity.vehicle.trip.start_time,
                            'start_date': entity.vehicle.trip.start_date
                            # Add other trip attributes as needed
                        },
                        'position': {
                            'latitude': entity.vehicle.position.latitude,
                            'longitude': entity.vehicle.position.longitude,
                            'bearing': entity.vehicle.position.bearing
                            # Add other trip attributes as needed
                        },
                        'position': {
                            'latitude': entity.vehicle.position.latitude,
                            'longitude': entity.vehicle.position.longitude,
                            'bearing': entity.vehicle.position.bearing,
                            'odometer': entity.vehicle.position.odometer,
                            'speed' : entity.vehicle.position.speed
                            # Add other trip attributes as needed
                        },
                        'stop_id': entity.vehicle.stop_id,
                        'current_status': entity.vehicle.current_status,
                        'timestamp': entity.vehicle.timestamp
                    }
                }


                # Append the entity dictionary to the list
                decoded_entities.append(entity_dict)

        # Convert the list of decoded entities to a JSON array
        json_array = json.dumps(entity_dict)
        # print(type(json_array))
        return json_array
    else:
        print(f"Error fetching coordinates: {response.status_code}")
        return None


def produce_kafka_messages():
    producer = KafkaProducer(bootstrap_servers=KAFKA_SERVER, 
                             api_version=(0,11,5),
                             value_serializer=lambda x: json.dumps(x).encode('utf-8'))

    while True:
        coordinates = fetch_coordinates()
        if coordinates:
            producer.send(TOPIC, value=coordinates)
            print(f"Message produced: {coordinates}")

        time.sleep(2)  # Wait for 5 seconds before fetching coordinates again
stack = []
def consume_kafka_messages():
    # Kafka server and topic information
    # Create a Kafka consumer
    consumer = KafkaConsumer(TOPIC,
                             bootstrap_servers=KAFKA_SERVER,
                             api_version=(0, 11, 5),
                             value_deserializer=lambda x: json.loads(x.decode('utf-8')))

    # Consume messages indefinitely
    for message in consumer:

        coordinates = json.loads(message.value)
        print(f"Message consumed: {coordinates}")
        
        # Example: Access individual fields in the coordinates

In [2]:
response = requests.get(api_url)
if response.status_code == 200:
        # Fetch the GTFS-RT payloal
        response = requests.get(api_url)
        payload = response.content

        # Create a GTFS-RT feed message
        feed = gtfs_realtime_pb2.FeedMessage()

        # Parse and decode the payload
        feed.ParseFromString(payload)

In [3]:
print(feed)

header {
  gtfs_realtime_version: "2.0"
  incrementality: FULL_DATASET
  timestamp: 1709887789
}
entity {
  id: "vehicle_position_22/943"
  vehicle {
    trip {
      route_id: "1057"
      direction_id: 1
      start_time: "10:12:00"
      start_date: "20240308"
      schedule_relationship: SCHEDULED
    }
    vehicle {
      id: "22/943"
    }
    position {
      latitude: 60.229744
      longitude: 25.0537739
      bearing: 53
      odometer: 13384
      speed: 4.37
    }
    stop_id: "1454133"
    current_status: STOPPED_AT
    timestamp: 1709887789
  }
}
entity {
  id: "vehicle_position_22/947"
  vehicle {
    trip {
      route_id: "2502"
      direction_id: 1
      start_time: "10:47:00"
      start_date: "20240308"
      schedule_relationship: SCHEDULED
    }
    vehicle {
      id: "22/947"
    }
    position {
      latitude: 60.2170296
      longitude: 24.8252735
      bearing: 91
      odometer: 1009
      speed: 0.1
    }
    stop_id: "2118278"
    current_status: STOPPED

In [6]:
produce_kafka_messages()

UnboundLocalError: local variable 'entity_dict' referenced before assignment

In [3]:
consume_kafka_messages()

KeyboardInterrupt: 

In [11]:
print(stack)

['{"id": "vehicle_position_47/668", "vehicle": {"trip": {"route_id": "1016", "start_time": "10:08:00", "start_date": "20240305"}, "position": {"latitude": 60.18048095703125, "longitude": 24.992950439453125, "bearing": 217.0, "odometer": 8070.0, "speed": 7.510000228881836}, "stop_id": "1420125", "current_status": 2, "timestamp": 1709627687}}']


In [15]:
json_data = json.loads(stack[0])

In [16]:
print(json_data)

{'id': 'vehicle_position_47/668', 'vehicle': {'trip': {'route_id': '1016', 'start_time': '10:08:00', 'start_date': '20240305'}, 'position': {'latitude': 60.18048095703125, 'longitude': 24.992950439453125, 'bearing': 217.0, 'odometer': 8070.0, 'speed': 7.510000228881836}, 'stop_id': '1420125', 'current_status': 2, 'timestamp': 1709627687}}
